### SERVER side

In [ ]:
import time, socket, sys
import threading


HOST = "0.0.0.0"
s = socket.socket ()
host = socket.gethostname ()
ip = socket.gethostbyname (host)
port = 8081
s.bind ((HOST, port))
print(ip)
FORMAT = "ASCII"
first_message="POST / HTTP/1.1\n"
clients = set()                    #to keep track of all the clients connected
clients_lock = threading.Lock()

def parse_req(lines):    
    method = lines[0].split('/')[0].strip(' ').upper()    
    if method != 'POST':
        raise Exception('HTTP/1.1 405 Method Not Allowed!')    
    i = lines.index('')
    headers = lines[:i]
    msgType = headers[-1].split(':')[1].strip(' ')
    messages = lines[i+1:]
    return msgType, messages, method, headers


def divide_mex(mex):
    """This functions divides the headers from the content of a message """
    n = 1
    for i in range(len(mex)):
        if mex[i]=="":
            n=i+1
    header = "\n".join(mex[:n])
    message = "\n".join(mex[n:])
    return header,message
    
    

def handle_client(clientSocket, clientAddress):
    
    try:
        connected = True
        name = ""
        while connected: 
            try:
                msg = clientSocket.recv(1024).decode(FORMAT)
                if not msg: 
                    continue
                typ, msg,_,_ = parse_req(msg.split('\n'))
                if typ == 'connect':
                    name = msg[0]
                    clientSocket.send("HTTP/1.1 200 OK\n"
                            "myline: connect"
                            "\n"
                            "\n"
                            "HELLO {} NICE TO MEET YOU".format(msg[0]).encode(FORMAT))
                    print("{} connected to the room.".format(msg[0]))
                    continue
                elif typ == 'message to send':
                    clientSocket.send("HTTP/1.1 200 OK\n"
                            "myline: message to send"
                            "\n"
                            "\n"
                            "MESSAGE ACCEPTED FOR DELIVERY".encode(FORMAT))
                    print('From: '+'\n'.join(msg))
                    with clients_lock:
                        notbroken = set()                    
                        for c in clients:
                            try:
                                if c != clientSocket:
                                    header = ("HTTP/1.1 200 OK\n"
                                             "myline: message to receive\n"                                     
                                             "\n"
                                             "From ")
                                    message = header + ('\n'.join(msg))
                                    c.sendall(message.encode(FORMAT))
                                notbroken.add(c)
                            except BrokenPipeError:
                                clients.remove(c)
                    continue
                elif typ == 'message to receive':
                    print('\n'.join(msg))
                    continue
                else:
                    break
            except ConnectionResetError:
                clients.remove(clientSocket)
                print("{} left the room.".format(name))
    finally:
        with clients_lock:
            clients.remove(clientSocket)
        clientSocket.close()
            
            
            
            
            
def start():
    s.listen(1)
    print ('The server is ready to receive')
    while True:                                     # it infinitely waits and try to accept incoming requests
        clientSocket, addr = s.accept()  
        with clients_lock:
            clients.add(clientSocket)      #this unsure us that clients is modified by a single thread each time, to have consistency
            """It deny other thread to modify that client"""
        thread = threading.Thread(target= handle_client , args = (clientSocket, addr)) 
        thread.start()
        """ Each time we have a connection we create a thread, which allows to not block clients while they are not 
        sending messages. 
        We want that while the server waits for others connection , other clients are also able to send messages. """
    





start()


172.20.10.4
The server is ready to receive
yaya connected to the room.
From: yaya
hi
From: yaya
how are you?


Exception in thread Thread-6:
Traceback (most recent call last):
  File "<ipython-input-1-2b9503a6d70a>", line 47, in handle_client
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-1-2b9503a6d70a>", line 88, in handle_client
KeyError: <socket.socket fd=1300, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('172.20.10.4', 8081), raddr=('172.20.10.3', 49938)>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Utente\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\Utente\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-2b9503a6d70a>", line 92, in handle_client
KeyError: <socket.socket fd=1300, family=Address

yaya left the room.
